In [1]:
%pip install spacy grecy

  Using cached spacy-3.8.5-cp312-cp312-macosx_11_0_arm64.whl.metadata (27 kB)
  Using cached grecy-1.0-py3-none-any.whl.metadata (5.0 kB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached murmurhash-1.0.12-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached cymem-2.0.11-cp312-cp312-macosx_11_0_arm64.whl.metadata (8.5 kB)
  Using cached preshed-3.0.9-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.2 kB)
  Using cached wasabi-1.1.3-py3-none-any.whl.metadata (28 kB)
  Using cached srsly-2.5.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (19 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl.metadata (14 kB)
  Using cached weasel-0.4.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached typer-0.15.2-py3-none-any.whl.metadata (15 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached numpy-2.2.4-cp312-cp312-macosx_14_0_arm64.whl.metadata (62 kB)
  U

In [1]:
%pip install https://huggingface.co/Jacobo/grc_proiel_trf/resolve/main/grc_proiel_trf-3.7.5-py3-none-any.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.5/497.5 MB 14.1 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip is available: 24.1 -> 25.0.1
[notice] To update, run: /Users/pletcher/code/writing/articles/2025-04-11_thucydidean-hedges/.venv/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install polars MyCapytain

  Using cached MyCapytain-3.0.2-py3-none-any.whl
  Using cached future-1.0.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached rdflib_jsonld-0.6.2-py2.py3-none-any.whl.metadata (3.4 kB)
  Using cached LinkHeader-0.4.3-py3-none-any.whl
  Using cached PyLD-2.0.4-py3-none-any.whl.metadata (10.0 kB)
  Using cached typing-3.7.4.3-py3-none-any.whl
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.6/31.6 MB 49.5 MB/s eta 0:00:00m eta 0:00:010:01:01
Using cached future-1.0.0-py3-none-any.whl (491 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 9.5 MB/s eta 0:00:00m eta 0:00:010:00:01
Using cached PyLD-2.0.4-py3-none-any.whl (70 kB)
Using cached rdflib_jsonld-0.6.2-py2.py3-none-any.whl (4.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 32.0 MB/s eta 0:00:00
Using cached cachetools-5.5.2-py3-none-any.whl (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.1/111.1 kB 10.6 MB/s eta 0:00:00

[notice] A n

In [64]:
import polars as pl

SPEECHES = pl.read_csv("./thuc-speeches.tsv", separator="\t").with_columns(
    pl.col("start", "end").str.split(".").cast(pl.List(pl.UInt32)),
    pl.int_range(pl.len(), dtype=pl.UInt32).alias("index"),
)


def get_speech_for_ref(ref: list[int]):
    for speech in SPEECHES.iter_rows(named=True):
        ref_is_after_speech_start = True
        ref_is_before_speech_end = True

        for pair in zip(speech["start"], ref):
            if pair[0] > pair[1]:
                ref_is_after_speech_start = False

        for pair in zip(speech["end"], ref):
            if pair[0] < pair[1]:
                ref_is_before_speech_end = False

        if ref_is_after_speech_start and ref_is_before_speech_end:
            return dict(
                id=speech["index"],
                speaker=speech["speaker"],
                location=speech["location"],
            )

    return None

In [78]:
from MyCapytain.resources.texts.local.capitains.cts import CapitainsCtsText
from MyCapytain.common.constants import Mimetypes

THUCYDIDES_PARQUET = "thucydides.parquet"

def thuc_to_table():
    """
    See https://mycapytain.readthedocs.io/en/latest/MyCapytain.local.html
    for a fuller example.
    """
    with open("./tei_primary_sources/tlg0003.tlg001.perseus-grc2.xml") as f:
        # We initiate a Text object giving the IO instance to resource argument
        text = CapitainsCtsText(resource=f)

    refs = [r[0] for r in text.getReffs(level=len(text.citation))]
    passages = [
        text.getTextualNode(subreference=ref, simple=True).export(
            Mimetypes.PLAINTEXT, exclude=["tei:note"]
        )
        for ref in refs
    ]
    refs = [list(map(lambda i: int(i), r.split("."))) for r in refs]
    speeches = [get_speech_for_ref(ref) for ref in refs]
    speech_ids = [s["id"] if s is not None else None for s in speeches]
    speakers = [s["speaker"] if s is not None else None for s in speeches]
    locations = [s["location"] if s is not None else None for s in speeches]

    return pl.DataFrame(
        {
            "reference": refs,
            "passage": passages,
            "speaker": speakers,
            "speech_id": speech_ids,
            "location": locations,
        }
    )

In [4]:
import spacy

SPACY_MODEL = "grc_proiel_trf"

nlp = spacy.load(SPACY_MODEL)

/Users/pletcher/code/writing/articles/2025-04-11_thucydidean-hedges/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def save_df(df: pl.DataFrame) -> pl.DataFrame:
    df = thuc_to_table().with_columns(
        pl.col("passage")
        .map_elements(lambda p: spacy.tokens.DocBin(docs=[nlp(p)]).to_bytes())
        .alias("parsed_passage")
    )

    df.write_parquet(THUCYDIDES_PARQUET)

In [86]:
def restore_df():
    return pl.read_parquet(THUCYDIDES_PARQUET).with_columns(
        pl.col("parsed_passage").map_elements(
            lambda p: list(spacy.tokens.DocBin().from_bytes(p).get_docs(nlp.vocab))[0])
    )

In [87]:
df = restore_df()

/var/folders/49/45tjf9pd6wj39p3hpxs1bm7r0000gn/T/ipykernel_64602/2957781896.py:2: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  return pl.read_parquet(THUCYDIDES_PARQUET).with_columns(


In [88]:
for token in df['parsed_passage'][1]:
    print((token, token.lemma_, token.pos_, token.dep_, token.morph.to_dict()))

(κίνησις, 'κίνησις', 'NOUN', 'nsubj', {'Case': 'Nom', 'Gender': 'Fem', 'Number': 'Sing'})
(γὰρ, 'γάρ', 'ADV', 'discourse', {})
(αὕτη, 'οὗτος', 'DET', 'det', {'Case': 'Nom', 'Gender': 'Fem', 'Number': 'Sing'})
(μεγίστη, 'μέγας', 'ADJ', 'amod', {'Case': 'Nom', 'Degree': 'Pos', 'Gender': 'Fem', 'Number': 'Sing'})
(δὴ, 'δή', 'ADV', 'discourse', {})
(τοῖς, 'ὁ', 'DET', 'det', {'Case': 'Dat', 'Definite': 'Def', 'Gender': 'Masc', 'Number': 'Plur', 'PronType': 'Dem'})
(Ἕλλησιν, 'Ἕλλην', 'NOUN', 'iobj', {'Case': 'Dat', 'Gender': 'Masc', 'Number': 'Plur'})
(ἐγένετο, 'γίγνομαι', 'VERB', 'ROOT', {'Aspect': 'Perf', 'Mood': 'Ind', 'Number': 'Sing', 'Person': '3', 'Tense': 'Past', 'VerbForm': 'Fin', 'Voice': 'Mid'})
(καὶ, 'καί', 'CCONJ', 'cc', {})
(μέρει, 'μέρος', 'NOUN', 'conj', {'Case': 'Dat', 'Gender': 'Neut', 'Number': 'Sing'})
(τινὶ, 'τίς', 'DET', 'det', {'Case': 'Dat', 'Gender': 'Masc', 'Number': 'Sing'})
(τῶν, 'ὁ', 'DET', 'det', {'Case': 'Gen', 'Definite': 'Def', 'Gender': 'Masc', 'Number': 'Pl